In game Win Probability Modelling:
============================================
Part 1 : Data Preprocessing, Feature Engineering
---------------------------------------------------

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from collections import deque
import codecs
import math

In [2]:
# Setting display priorities, might cause security issues because of the raw html injection !
from IPython.core.display import display, HTML
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
display(HTML("<style>.container { width:80% !important; }</style>")) # comment for security issues
np.set_printoptions(precision=4)
pd.set_option('display.max_columns',50)

Data Preprocessing and Feature Engineering:
---------------------------------------------

Most of the features have to extracted from the data points by using regular expression.
A detailed list of final features and their description will be available in the file, `feature_dictionary.txt`  
The features are seperated into three families,  

1) **In game features** : Features that are extracted from the current data records and the records that occured before it  
2) **In season features** : Features that are extracted from previous games of the same regualar season  
3) **Pre Season features (External Data)** : External data from the previous year regular season performance. For 2016 I used 2014-2015 regular season hollinger stats  and for 2017 games I used 2015-2016 regualar season [hollinger stats](http://www.espn.com/nba/hollinger/teamstats)  
4) **Player features**: [Hollinger stats for players](http://insider.espn.com/nba/hollinger/statistics) is used in this segment. 


In [3]:
# Reading data
data = pd.read_csv("pbp.csv")
len(np.unique(data['game_id'].values))
data.shape

(294366, 18)

From the score field we extract `score_difference` and `score_diff_percentile`
and we convert the time to seconds as `remaining_time`

$$score\ difference = away\ score - home\ score $$
$$score\ difference\ percentile = \frac{away\ score  - home\ score}{away\ score + home\ score}$$

In [4]:
# Remaining Time in seconds
temp = data['play_clock'].str.extract("(\d+):(\d+)")
data_clean = data.copy()
data_clean['remaining_time'] = temp[0].astype('int32')*60 + temp[1].astype('int32')

# Score Difference between the teams
before = ''
score = data_clean['score']
for i in range(len(score)):
    if type(score[i]) is str:
        before = score[i]
    else:
        score.set_value(i, before)
        
temp = data_clean['score'].str.extract("(\d+) - (\d+)")
data_clean['score_difference'] = temp[0].astype('int32') - temp[1].astype('int32')
data_clean['score_diff_percentile'] = (temp[0].astype('float32') - temp[1].astype('int32'))/(temp[0].astype('int32') + temp[1].astype('int32'))
data_clean.head()

C:\Users\Adiamaan Keerthi\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app
C:\Users\Adiamaan Keerthi\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


,sequence_id,game_id,period,play_clock,home_description,away_description,score,player1_id,player1_name,player1_team,player2_id,player2_name,player2_team,player3_id,player3_name,player3_team,event_type,event_description,remaining_time,score_difference,score_diff_percentile
0,0,21500001,4,12:00,NaN,NaN,82 - 66,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,Start Period,NaN,720,16,0.108108
1,1,21500001,4,11:44,NaN,Baynes Dunk (2 PTS) (Ilyasova 3 AST),84 - 66,203382,Aron Baynes,DET,101141,Ersan Ilyasova,DET,0,NaN,NaN,Made Shot,NaN,704,18,0.120000
2,2,21500001,4,11:29,Schroder 2' Driving Finger Roll Layup (16 PTS),NaN,84 - 68,203471,Dennis Schroder,ATL,0,NaN,NaN,0,NaN,NaN,Made Shot,NaN,689,16,0.105263
3,3,21500001,4,11:17,Schroder STEAL (2 STL),Blake Lost Ball Turnover (P3.T13),84 - 68,2581,Steve Blake,DET,203471,Dennis Schroder,ATL,0,NaN,NaN,Turnover,Lost Ball,677,16,0.105263
4,4,21500001,4,11:14,NaN,Ilyasova S.FOUL (P3.T1) (Z.Zarba),84 - 68,101141,Ersan Ilyasova,DET,203471,Dennis Schroder,ATL,0,NaN,NaN,Foul,Shooting,674,16,0.105263


For each game we find the `home` and `away` team from home and away description  
and using the event type "end game" we set the `home_result` to 0 or 1.  
These data is then propogated to all records of the particular game so that each point can act as  
seperate data point for modelling.

In [5]:
# Initializing fields
data_clean['home_result'] = pd.Series(dtype='int32')
data_clean['home'] = pd.Series(dtype='object')
data_clean['away'] = pd.Series(dtype='object')

# Calculating the home, away team and home team result
game_set = dict()
home, away = '', ''
home_f, away_f = False, False
for i in range(data_clean.shape[0]):
    game_id = data_clean['game_id'][i]
    if game_id not in game_set and game_id not in [21500586]:
        name = data_clean['player1_name'][i]
        if type(name) is str:
            name = name.split(' ')
            name = name[len(name) - 1]
            if name in data_clean['home_description'] and name in data_clean['away_description']:
                print(game_id)
                game_set[game_id] = ('Locked')
            elif not home_f and type(data_clean['home_description'][i]) is str:
                if name in data_clean['home_description'][i]:
                    home = data_clean['player1_team'][i]
                    home_f = True                       
            elif not away_f and type(data_clean['away_description'][i]) is str:
                if name in data_clean['away_description'][i]:
                    away = data_clean['player1_team'][i]
                    away_f = True
        if home_f and away_f and data_clean['remaining_time'][i] == 0 and away != home:
            match = re.match(r'(\d+) - (\d+)',data_clean['score'][i])
            away_score, home_score = int(match.group(1)), int(match.group(2))
            home_result = 0 if away_score > home_score else 1
            print(game_id, home, away, home_score, away_score, home_result)
            home_f, away_f = False, False
            game_set[game_id] = (home, away, home_score, away_score, home_result)

21500001 ATL DET 94 106 0
21500002 CHI CLE 97 95 1
21500003 GSW NOP 111 95 1
21500004 ORL WAS 87 88 0
21500005 BOS PHI 112 95 1
21500006 BKN CHI 100 115 0
21500007 DET UTA 92 87 1
21500008 MIA CHA 104 94 1
21500009 TOR IND 106 99 1
21500010 HOU DEN 85 105 0
21500011 MEM CLE 76 106 0
21500012 MIL NYK 97 122 0
21500013 OKC SAS 112 106 1
21500014 PHX DAL 95 111 0
21500015 POR NOP 112 94 1
21500016 SAC LAC 104 110 0
21500017 LAL MIN 111 112 0
21500018 IND MEM 103 112 0
21500019 NYK ATL 101 112 0
21500020 LAC DAL 104 88 1
21500021 CLE MIA 102 92 1
21500022 ORL OKC 117 117 1
21500023 PHI UTA 71 99 0
21500024 BOS TOR 103 113 0
21500025 DET CHI 83 83 1
21500026 ATL CHA 97 94 1
21500027 MIL WAS 113 118 0
21500028 SAS BKN 102 75 1
21500029 DEN MIN 78 95 0
21500030 HOU GSW 92 112 0
21500031 SAC LAL 132 114 1
21500032 PHX POR 110 92 1
21500033 IND UTA 76 97 0
21500034 WAS NYK 110 117 0
21500035 NOP GSW 120 134 0
21500036 MEM BKN 101 91 1
21500037 POR PHX 90 101 0
21500038 LAC SAC 114 109 1
2150003

21500326 PHX ORL 107 104 1
21500327 UTA NYK 106 85 1
21500328 DAL ATL 95 98 0
21500329 BKN PHI 100 91 1
21500330 CHI LAC 83 80 1
21500331 OKC ATL 107 94 1
21500332 SAC NYK 99 97 1
21500333 IND MIA 96 83 1
21500334 ORL CLE 76 111 0
21500335 PHI DET 95 107 0
21500336 BOS GSW 103 103 1
21500337 TOR MIL 90 83 1
21500338 MEM CHA 99 123 0
21500339 NOP WAS 107 105 1
21500340 DEN MIN 100 100 1
21500341 UTA OKC 90 94 0
21500342 SAS LAL 109 87 1
21500343 PHX POR 96 106 0
21500344 BKN LAC 100 105 0
21500345 CHA BOS 93 98 0
21500346 DET IND 118 96 1
21500347 ATL SAS 78 103 0
21500348 CHI NOP 98 94 1
21500349 HOU LAL 126 97 1
21500350 DAL WAS 111 114 0
21500351 MIL GSW 108 95 1
21500352 POR NYK 110 112 0
21500353 PHX MIN 108 101 1
21500354 MIA MEM 98 97 1
21500355 TOR PHI 96 76 1
21500356 OKC UTA 96 96 1
21500357 IND TOR 106 90 1
21500358 BKN ORL 82 105 0
21500359 DET LAC 94 94 1
21500360 ATL MIA 88 100 0
21500361 CHI PHI 115 96 1
21500362 MEM WAS 112 95 1
21500363 DAL PHX 104 94 1
21500364 SAS UTA

21500654 WAS UTA 103 89 1
21500655 CHA NYK 97 84 1
21500656 PHI BOS 92 112 0
21500657 NOP MIL 116 99 1
21500658 MIN MEM 106 101 1
21500659 CLE CHI 83 96 0
21500660 PHX ATL 98 95 1
21500661 DEN DET 102 101 1
21500662 POR LAL 121 103 1
21500663 SAC IND 108 97 1
21500664 BKN OKC 116 106 1
21500665 HOU DAL 115 104 1
21500666 TOR LAC 112 94 1
21500667 CLE MIN 114 107 1
21500668 WAS BOS 91 116 0
21500669 CHI MIA 84 89 0
21500670 MEM ORL 100 100 1
21500671 NOP HOU 111 112 0
21500672 DEN ATL 105 119 0
21500673 UTA DET 92 95 0
21500674 SAC CHA 115 114 1
21500675 GSW SAS 120 90 1
21500676 IND LAC 89 91 0
21500677 PHI PHX 113 103 1
21500678 BKN MIA 98 102 0
21500679 NYK OKC 113 113 1
21500680 TOR WAS 106 89 1
21500681 MIL ORL 107 100 1
21500682 POR SAC 112 97 1
21500683 LAL DAL 90 92 0
21500684 CLE PHX 115 93 1
21500685 BOS DEN 111 103 1
21500686 DET PHI 110 97 1
21500687 ATL LAC 83 85 0
21500688 MIN OKC 123 126 0
21500689 SAS HOU 130 99 1
21500690 UTA CHA 102 73 1
21500691 GSW DAL 127 107 1
2150

21500988 CHA DAL 96 107 0
21500989 WAS DET 124 81 1
21500990 MIA DEN 124 119 1
21500991 TOR CHI 107 109 0
21500992 HOU MEM 130 81 1
21500993 OKC POR 128 94 1
21500994 UTA CLE 94 85 1
21500995 PHX MIN 107 104 1
21500996 GSW NOP 125 107 1
21500997 IND BOS 103 98 1
21500998 ORL DEN 116 110 1
21500999 BKN PHI 131 114 1
21501000 MIL TOR 89 107 0
21501001 SAS LAC 108 87 1
21501002 LAL SAC 98 106 0
21501003 BOS OKC 109 130 0
21501004 CHA ORL 107 99 1
21501005 CLE DAL 99 98 1
21501006 WAS CHI 117 96 1
21501007 DET ATL 114 118 0
21501008 MEM MIN 108 114 0
21501009 HOU LAC 106 122 0
21501010 SAC NOP 108 123 0
21501011 GSW NYK 121 85 1
21501012 IND TOR 85 85 1
21501013 PHI WAS 94 97 0
21501014 MIA CHA 106 109 0
21501015 ATL DEN 116 98 1
21501016 CHI BKN 118 102 1
21501017 MIL MEM 96 86 1
21501018 SAS POR 118 110 1
21501019 UTA PHX 103 69 1
21501020 ORL CLE 103 109 0
21501021 PHI OKC 97 111 0
21501022 DET SAC 115 108 1
21501023 TOR BOS 105 91 1
21501024 HOU MIN 116 111 1
21501025 NOP POR 112 117 0

21600094 WAS HOU 106 114 0
21600095 CHA IND 122 100 1
21600096 CHI ORL 112 80 1
21600097 OKC MIA 97 85 1
21600098 LAC DET 114 82 1
21600099 GSW NOP 116 106 1
21600100 CLE ATL 106 110 0
21600101 BKN MIN 119 110 1
21600102 MEM DEN 106 107 0
21600103 POR PHX 124 121 1
21600104 LAL DAL 97 109 0
21600105 SAC NOP 102 94 1
21600106 NYK BKN 110 96 1
21600107 WAS BOS 118 93 1
21600108 CHA UTA 104 98 1
21600109 ORL MIN 107 123 0
21600110 IND PHI 109 109 1
21600111 ATL CHI 115 107 1
21600112 OKC TOR 102 112 0
21600113 PHX DET 107 100 1
21600114 SAS HOU 99 101 0
21600115 LAC POR 111 80 1
21600116 GSW DAL 116 95 1
21600117 MIL NOP 106 112 0
21600118 MIA CHI 95 98 0
21600119 DEN GSW 101 125 0
21600120 SAC LAL 91 101 0
21600121 PHI IND 100 100 1
21600122 WAS CLE 94 105 0
21600123 CHA TOR 111 113 0
21600124 ORL UTA 74 87 0
21600125 BOS NYK 115 87 1
21600126 OKC LAC 108 110 0
21600127 SAS DET 96 86 1
21600128 POR SAC 106 106 1
21600129 NOP LAL 99 126 0
21600130 IND BOS 99 105 0
21600131 TOR NYK 118 107

21600425 HOU SAS 100 102 0
21600426 LAC DEN 119 102 1
21600427 GSW UTA 104 74 1
21600428 SAC POR 124 121 1
21600429 CLE MIL 113 102 1
21600430 ATL MIN 84 92 0
21600431 DET MEM 86 98 0
21600432 CHI WAS 97 107 0
21600433 NOP OKC 110 121 0
21600434 UTA SAC 93 94 0
21600435 PHX HOU 111 125 0
21600436 POR DAL 95 96 0
21600437 BKN GSW 101 117 0
21600438 NYK ORL 106 95 1
21600439 MIA LAL 115 107 1
21600440 IND BOS 102 109 0
21600441 LAC SAS 106 101 1
21600442 CHA CHI 103 91 1
21600443 ORL LAL 109 90 1
21600444 CLE BKN 119 99 1
21600445 BOS OKC 112 117 0
21600446 DET GSW 113 119 0
21600447 MIL WAS 123 96 1
21600448 MEM HOU 115 109 1
21600449 NOP MIA 91 87 1
21600450 MIN SAC 105 109 0
21600451 DEN ATL 108 109 0
21600452 UTA TOR 98 104 0
21600453 PHX PHI 123 116 1
21600454 POR SAS 90 110 0
21600455 LAC DAL 88 90 0
21600456 NYK BOS 114 119 0
21600457 CLE GSW 109 108 1
21600458 SAS CHI 119 100 1
21600459 OKC MIN 112 100 1
21600460 LAL LAC 111 102 1
21600461 WAS MIL 107 102 1
21600462 ORL MEM 112 1

21600745 LAC GSW 120 133 0
21600746 ORL TOR 102 94 1
21600747 BKN IND 97 106 0
21600748 DET MIN 116 108 1
21600749 HOU CHI 108 108 1
21600750 OKC MEM 114 102 1
21600751 BOS LAL 113 107 1
21600752 DEN MIL 121 117 1
21600753 SAC PHX 103 105 0
21600754 POR DAL 104 108 0
21600755 WAS NOP 105 91 1
21600756 ATL ORL 113 86 1
21600757 IND DET 105 84 1
21600758 MIA PHI 125 102 1
21600759 NYK CLE 104 111 0
21600760 UTA CHA 105 98 1
21600761 PHX MIL 112 137 0
21600762 SAS DEN 121 97 1
21600763 MIN MEM 99 107 0
21600764 SAC GSW 98 98 1
21600765 BKN TOR 95 103 0
21600766 BOS LAC 107 102 1
21600767 OKC POR 105 99 1
21600768 NYK LAL 107 121 0
21600769 WAS CLE 120 120 1
21600770 IND OKC 93 90 1
21600771 TOR LAC 118 109 1
21600772 ATL UTA 95 120 0
21600773 DET PHI 113 96 1
21600774 NOP PHX 111 106 1
21600775 MIN MIA 113 115 0
21600776 DEN DAL 110 87 1
21600777 MEM SAS 89 74 1
21600778 SAC CHI 107 112 0
21600779 CHA BKN 111 107 1
21600780 HOU ORL 128 104 1
21600781 DAL POR 113 114 0
21600782 PHI SAS 103

21601067 BKN PHX 126 98 1
21601068 MIA TOR 84 101 0
21601069 DAL LAC 96 95 1
21601070 SAS MEM 97 90 1
21601071 POR NYK 110 95 1
21601072 WAS BKN 129 108 1
21601073 CHA CLE 105 112 0
21601074 ORL DET 115 87 1
21601075 IND DEN 117 125 0
21601076 BOS PHX 130 120 1
21601077 CHI PHI 107 117 0
21601078 MIL ATL 100 97 1
21601079 HOU NOP 117 107 1
21601080 LAL MIN 109 109 1
21601081 GSW SAC 114 100 1
21601082 LAC UTA 108 95 1
21601083 CLE WAS 115 127 0
21601084 DAL TOR 86 94 0
21601085 SAS NYK 106 98 1
21601086 POR MIN 112 100 1
21601087 CHA PHX 120 106 1
21601088 ATL BKN 92 107 0
21601089 LAC SAC 97 98 0
21601090 MIL CHI 94 109 0
21601091 HOU OKC 137 125 1
21601092 IND PHI 107 94 1
21601093 BOS MIA 112 108 1
21601094 GSW MEM 106 94 1
21601095 DEN NOP 90 115 0
21601096 LAL POR 81 97 0
21601097 TOR ORL 131 112 1
21601098 NYK DET 109 95 1
21601099 SAS CLE 103 74 1
21601100 DAL OKC 91 92 0
21601101 SAC MEM 91 90 1
21601102 UTA NOP 108 100 1
21601103 CHA MIL 108 118 0
21601104 IND MIN 114 115 0
21

In [6]:
# A rare game where a player with a last name 'Johnson' appears in both away and home description 
# making it tough to find home and away team. Manually adding the record as a workaround
game_set[21500586] = ('LAC', 'MIA', 104, 90, 1)

In [7]:
# Propogating the information to all the records
for i in range((data_clean.shape[0])):
    record = game_set[data_clean['game_id'][i]]
    data_clean['home'].set_value(i, record[0])
    data_clean['away'].set_value(i, record[1])
    data_clean['home_result'].set_value(i, record[4])

In game features are created by looking at the records that appeared before it in the same game.
Features like `in_violations`, `in_rebound`, `in_turnover` are calculated as  

    in_feature = home_feature - away_feature  
    
If the feature is violation then we subtract the home_violations so far minus away    violations so far. This is better than having a seperate feature for each team. It keeps the    data dense and reduces the dimensionality as well

    `in_shot_accuracy` is calculated as follows,
$$ in\ shot\ accuracy = \frac{home\ Made\ Shot}{(home\ Made\ Shot)+(home\ Missed\ Shot} - \frac{away\ Made\ Shot}{(away\ Made\ Shot)+(away\ Missed\ Shot)}$$

In [8]:
# Creating fields for In game features
data_clean['in_violations'] = pd.Series(dtype='int32')
data_clean['in_rebound'] = pd.Series(dtype='int32')
data_clean['in_hfouls'] = pd.Series(dtype='int32')
data_clean['in_hshoot_fouls'] = pd.Series(dtype='int32')
data_clean['in_afouls'] = pd.Series(dtype='int32')
data_clean['in_ashoot_fouls'] = pd.Series(dtype='int32')
data_clean['in_turnover'] = pd.Series(dtype='int32')
data_clean['in_shot_accuracy'] = pd.Series(dtype='int32')
data_clean['in_ft_accuracy'] = pd.Series(dtype='int32')
data_clean['in_momentum'] = pd.Series(dtype='int32')

In [9]:
data_clean['event_type'] = data_clean['event_type'].str.strip()
game_after = 0
for i in range(data_clean.shape[0]):
    game_before = data_clean['game_id'][i]
    if game_before != game_after:
        in_violations, in_rebound, in_fouls, in_turnover, in_shoot_fouls, in_shot_accuracy, in_ft_accuracy = 0, 0, 0, 0, 0, 0, 0
        made_home, missed_home, made_away, missed_away, home_viol, away_viol, home_rebound, away_rebound = 0, 0, 0, 0, 0, 0, 0, 0
        home_sfoul, away_sfoul, home_foul, away_foul, home_ft_success, away_ft_success, home_ft_fail, away_ft_fail = 0, 0, 0, 0, 0, 0, 0, 0
        home_to, away_to = 0, 0
        start_diff = data_clean['game_id'][i]
        game_after = game_before
        momentum = deque([0]*5, maxlen=5)
    else:
        event = data_clean['event_type'][i]
        event_desc = data_clean['event_description'][i]
        home_desc = '' if type(data_clean['home_description'][i]) is float else data_clean['home_description'][i]
        away_desc = '' if type(data_clean['away_description'][i]) is float else data_clean['away_description'][i]
        momentum.append(data_clean['score_difference'][i])
        
        if event == 'Made Shot':
            made_home = (made_home+1) if type(home_desc) is str else made_home
            made_away = (made_away+1) if type(away_desc) is str else made_away
        elif event == 'Missed Shot':
            missed_home = (missed_home+1) if 'miss' in home_desc.lower() else missed_home
            missed_away = (missed_away+1) if 'miss' in away_desc.lower() else missed_away
        elif event == 'Violation':
            home_viol = (home_viol+1) if 'Violation' in home_desc else home_viol
            away_viol = (away_viol+1) if 'Violation' in away_desc else away_viol
        elif event == 'Rebound':
            home_rebound = (home_rebound+1) if 'rebound' in home_desc.lower() else home_rebound
            away_rebound = (away_rebound+1) if 'rebound' in away_desc.lower() else away_rebound
        elif event == 'Turnover':
            home_to = (home_to+1) if 'turnover' in home_desc.lower() else home_to
            away_to = (away_to+1) if 'turnover' in away_desc.lower() else away_to
        elif event == 'Foul':
            if 'shooting' in event_desc.lower():
                home_sfoul = (home_sfoul+1) if 'foul' in home_desc.lower() else home_sfoul
                away_sfoul = (away_sfoul+1) if 'foul' in away_desc.lower() else away_sfoul
            else:
                home_foul = (home_foul+1) if 'foul' in home_desc.lower() else home_foul
                away_foul = (away_foul+1) if 'foul' in away_desc.lower() else away_foul
        elif event == 'Free Throw':
            if 'free throw' in home_desc.lower():
                if 'miss' in home_desc.lower():
                    home_ft_fail += 1
                else:
                    home_ft_success += 1
            elif 'free throw' in away_desc.lower():
                if 'miss' in away_desc.lower():
                    away_ft_fail += 1
                else:
                    away_ft_success += 1

        if (home_ft_fail == 0 and home_ft_success == 0):
            a = 0
        else:
            a = (home_ft_success/(home_ft_fail+home_ft_success))
        if (away_ft_fail == 0 and away_ft_success == 0):
            b = 0
        else:
            b = (away_ft_success/(away_ft_fail+away_ft_success))
        ft_accuracy =  a - b 
        data_clean['in_ft_accuracy'].set_value(i, ft_accuracy) 
        
        if (made_home == 0 and missed_home == 0):
            a = 0
        else:
            a = (made_home/(made_home+missed_home))
        if (made_away == 0 and missed_away == 0):
            b = 0
        else:
            b = (made_away/(missed_away+made_away))
        
        shot_accuracy =  a - b 
        data_clean['in_shot_accuracy'].set_value(i, shot_accuracy)
        
        if momentum[4] != 0 and momentum[0] != 0 and momentum[0] != momentum[4]:
            data_clean['in_momentum'].set_value(i, (momentum[4] - momentum[0])/(momentum[4] + momentum[0]))
        else:
            data_clean['in_momentum'].set_value(i, 0)
        data_clean['in_violations'].set_value(i, (home_viol - away_viol))
        data_clean['in_rebound'].set_value(i, (home_rebound - away_rebound))
        data_clean['in_turnover'].set_value(i, (home_to - away_to))
        data_clean['in_ashoot_fouls'].set_value(i, away_sfoul)
        data_clean['in_afouls'].set_value(i, away_foul)
        data_clean['in_hshoot_fouls'].set_value(i, home_sfoul)
        data_clean['in_hfouls'].set_value(i, home_foul)
        game_after = game_before

C:\Users\Adiamaan Keerthi\Anaconda3\lib\site-packages\ipykernel\__main__.py:78: RuntimeWarning: divide by zero encountered in int_scalars



The data set has 1230 total games for 2016 season and 1150 total games for 2017 season

$$ nth\ game\ in\ season = \frac{Nth\ game\ of\ the\ season}{Total\ number\ of\ games\ in\ the\ season}$$

$$ Total\ win\ ratio = \frac{No\ of\ games\ won\ so\ far}{No\ of\ games\ played\ so\ far}$$

$$ Home\ Away\ win\ ratio = \frac{No\ of\ games\ won\ so\ far\ against\ away\ team}{No\ of\ games\ played\ so\ far\ against\ away\ team}$$

In [10]:
# Creating fields to store in-season features
data_clean['total_win_ratio'] = pd.Series(dtype='float32')
data_clean['home_away_win_ratio'] = pd.Series(dtype='float32')
data_clean['nth_game_season'] = pd.Series(dtype='float32')

# Converting game_set into dataframe for easier data manipulation
game_set = pd.DataFrame.from_dict(game_set, orient='index')
game_set.reset_index(inplace=True)
game_set.columns = ['game_id', 'home', 'away', 'home_score', 'away_Score', 'home_result']
game_set['total_win_ratio'] = pd.Series(dtype='float32')
game_set['home_away_win_ratio'] = pd.Series(dtype='float32')
game_set['nth_game_season'] = pd.Series(dtype='float32')
game_set.head()

,game_id,home,away,home_score,away_Score,home_result,total_win_ratio,home_away_win_ratio,nth_game_season
0,21500001,ATL,DET,94,106,0,NaN,NaN,NaN
1,21500002,CHI,CLE,97,95,1,NaN,NaN,NaN
2,21500003,GSW,NOP,111,95,1,NaN,NaN,NaN
3,21500004,ORL,WAS,87,88,0,NaN,NaN,NaN
4,21500005,BOS,PHI,112,95,1,NaN,NaN,NaN


In [12]:
# Setting in-season features total_win_ratio and home_away_win ratio
year_before, year_after = 0, 2015
teams_list = np.unique(game_set['home']).tolist()
vs_stats = dict()
overall_stats = dict()

for i in range(game_set.shape[0]):
    year_before = 2015 if (str(game_set['game_id'][i])).startswith('215') else 2016
    if year_before != year_after:
        vs_stats = dict()
        overall_stats = dict()
        year_after = year_before
    game_id = game_set['game_id'][i]
    home = game_set['home'][i]
    away = game_set['away'][i]
    result = game_set['home_result'][i]
    m = re.match(r'\d{4}(\d{4})', str(game_id))
    season_game = int(m.group(1))
    team_list = [home, away]
    team_list.sort()

    if overall_stats.get(home) == None:
        overall_stats[home] = (0, 0)
    if overall_stats.get(away) == None:
        overall_stats[away] = (0, 0)
    hwin, htotal = overall_stats[home]
    awin, atotal = overall_stats[away]
    total_games = 1230
    
    game_set['nth_game_season'].set_value(i, (season_game/total_games))
    if htotal == 0:
        game_set['total_win_ratio'].set_value(i, 0)
    else:
        game_set['total_win_ratio'].set_value(i, (hwin / htotal))
    htotal, atotal = htotal + 1, atotal + 1
    if result == 1:
        hwin += 1
    else:
        awin += 1

    overall_stats[home] = (hwin, htotal)
    overall_stats[away] = (awin, atotal)
    key = team_list[0] + team_list[1]

    if vs_stats.get(key) == None:
        vs_stats[key] = (0, 0)
    hvwin, hvtotal = vs_stats[key]
    if hvtotal == 0:
        game_set['home_away_win_ratio'].set_value(i, 0)
    else:
        game_set['home_away_win_ratio'].set_value(i, hvwin / hvtotal)

    hvtotal += 1
    hvwin = (hvwin + 1) if result == 1 else hvwin
    vs_stats[key] = (hvwin, hvtotal)


In [13]:
# Copying game data from game_set to data_clean dataframe
for i in range(game_set.shape[0]):
    game_id = game_set['game_id'][i]
    total_win_ratio = game_set['total_win_ratio'][i]
    home_away_win_ratio = game_set['home_away_win_ratio'][i]
    nth_game_season = game_set['nth_game_season'][i]
    data_clean.ix[data_clean['game_id'] == game_id, 'total_win_ratio'] = total_win_ratio
    data_clean.ix[data_clean['game_id'] == game_id, 'home_away_win_ratio'] = home_away_win_ratio
    data_clean.ix[data_clean['game_id'] == game_id, 'nth_game_season'] = nth_game_season

[Hollinger team stats](http://www.espn.com/nba/hollinger/teamstats) for the regular season is used as the external data.
All the pre season features have a 'pre' prefix.  

All the external features are calculated as follows,
        
        pre_PACE = PACE of home team - PACE of away team
The same goes for all other features in this segment as well.

External features and their description,

        PACE = the number of possessions a team uses per game.
        ORR = Offensive Rebound Rate
        DRR = Defensive Rebound Rate
        EFF FG% = Effective Field Goal Percentage
        OFF EFF = Offensive Efficiency, Number of points a team scores per 100 possession
        DEF EFF = Defensive Efficiency, Number of points a team allows per 100 possession
.

    Assist Ratio,
$$AST = \frac{(Assists * 100)}{[(FGA + (FTA * 0.44) + Assists + Turnovers)]}$$

    Turnover Ratio,
$$TO = \frac{(Turnover * 100}{ [(FGA + (FTA x 0.44) + Assists + Turnovers]}$$

    Rebound Rate,
$$REBR = \frac{Rebounds * Team Minutes}{Player Minutes *( Team Rebounds + Opponent Rebounds)}$$

    True Shooting Percentage,
$$TS = \frac{Total Points * 50}{[(FGA + (FTA * 0.44)]}$$

In [14]:
# Read in the external features
external = pd.read_csv('team_ratings.csv')

def external_ratings(home, away, year):
    return (external[((external['TEAM_ABBR'] == home) & (external['YEAR'] == year))].values[0][3:] - external[((external['TEAM_ABBR'] == away) & (external['YEAR'] == year))].values[0][3:])


In [15]:
# Creating columns for external features
# pre means stats got before the game from last regular season

data_clean['pre_PACE'] = pd.Series(dtype='int32')
data_clean['pre_AST'] = pd.Series(dtype='int32')
data_clean['pre_TO'] = pd.Series(dtype='int32')
data_clean['pre_ORR'] = pd.Series(dtype='int32')
data_clean['pre_DRR'] = pd.Series(dtype='int32')
data_clean['pre_EFF_FG'] = pd.Series(dtype='int32')
data_clean['pre_TS'] = pd.Series(dtype='int32')
data_clean['pre_OFF_EFF'] = pd.Series(dtype='int32')
data_clean['pre_DEF_EFF'] = pd.Series(dtype='int32')

In [16]:
game_before, game_after = 0, 0
record = 0
for i in range(data_clean.shape[0]):
    game_before = data_clean['game_id'][i]
    if game_before != game_after:
        year = 2015 if (re.match(r'215\d\d\d\d\d', str(game_before)) != None) else 2016
        record = external_ratings(home=data_clean['home'][i], away=data_clean['away'][i], year=year)
        game_after = game_before
    data_clean['pre_PACE'].set_value(i, record[0])
    data_clean['pre_AST'].set_value(i, record[1])
    data_clean['pre_TO'].set_value(i, record[2])
    data_clean['pre_ORR'].set_value(i, record[3])
    data_clean['pre_DRR'].set_value(i, record[4])
    data_clean['pre_EFF_FG'].set_value(i, record[5])
    data_clean['pre_TS'].set_value(i, record[6])
    data_clean['pre_OFF_EFF'].set_value(i, record[7])
    data_clean['pre_DEF_EFF'].set_value(i, record[8])

The three player information are taken across all record and a dictionary if unique players are formed.
Two external data files are read into prating15 and prating16 to store in the player rating of season 2015 and 2016

In [17]:
# Grouping all player information
player_details = pd.DataFrame(data_clean.groupby(['player1_id', 'player1_name','player1_team']).size())
player_details = player_details.append(pd.DataFrame(data_clean.groupby(['player2_id', 'player2_name', 'player2_team']).size()))
player_details = player_details.append(pd.DataFrame(data_clean.groupby(['player3_id', 'player3_name', 'player3_team']).size()))
player_details.reset_index(inplace=True)
player_details.drop_duplicates(subset=['player_id'], inplace=True)
player_details.reset_index(drop=True, inplace=True)
player_details.columns = ['ID', 'PLAYER', 'TEAM', 'APPEARED']
player_details


In [19]:
#Reading external data files
with codecs.open("player_ratings_2015.csv", "r",encoding='utf-8', errors='ignore') as fdata:
    prating15 = pd.read_csv(fdata)

with codecs.open("player_ratings_2016.csv", "r",encoding='utf-8', errors='ignore') as fdata:
    prating16 = pd.read_csv(fdata)
    
merged15 = player_details.merge(prating14, on='PLAYER', how='outer')
merged16 = player_details.merge(prating15, on='PLAYER', how='outer')

In [20]:
def get_rank(pname, year):
    rank = 0
    if year == 2015 and merged15.PLAYER.isin([pname]).any():
        rank = merged14[merged15.PLAYER == pname].RK.values[0]
    elif year == 2016 and merged16.PLAYER.isin([pname]).any():
        rank = merged15[merged16.PLAYER == pname].RK.values[0]
    return 0 if math.isnan(rank) else rank

data_clean['home_rank'] = pd.Series(dtype='float32')
data_clean['away_rank'] = pd.Series(dtype='float32')

In [40]:
game_before, game_after = 0, 0
for i in range(data_clean.shape[0]):
    game_before = data_clean['game_id'][i]
    if game_before != game_after:
        print("Games loaded...{}".format(game_before), end='\r')
        home_rank, away_rank, home_cnt, away_cnt = 0, 0, 0, 0
        home, away = data_clean.home[i], data_clean.away[i]
        game_after = game_before
        home_dict, away_dict = dict(), dict()
    else:
        year = 2015 if (re.match(r'215\d\d\d\d\d', str(data_clean.game_id[i])) != None) else 2016
        event = data_clean['event_type'][i]
        if event == 'Substitution':
            playing = data_clean['player2_name'][i]
            bench = data_clean['player1_name'][i]
            sub_team = data_clean['player1_team'][i]
            playing_rank = get_rank(playing, year)
            bench_rank = get_rank(bench, year)
            dic = home_dict if sub_team == home else away_dict
            if playing_rank != 0:
                dic[playing] = playing_rank
            if bench_rank != 0 and bench in dic:
                del dic[bench]
        else:
            for k in range(1,4):
                name = "player" + str(k) + "_name"
                team = "player" + str(k) + "_team"
                pname = data_clean[name][i]
                pteam = data_clean[team][i]
                dic = home_dict if pteam == home else away_dict
                if pname not in dic:
                    rank = get_rank(pname, year)
                    if rank != 0:
                        dic[pname] = rank
        home_rank = sum([v for i,v in home_dict.items()])/max(len(home_dict), 1)
        away_rank = sum([v for i,v in away_dict.items()])/max(len(away_dict), 1)
        data_clean['home_rank'].set_value(i, home_rank)
        data_clean['away_rank'].set_value(i, away_rank)

Games loaded...21500001
Games loaded...21500002
Games loaded...21500003
Games loaded...21500004
Games loaded...21500005
Games loaded...21500006
Games loaded...21500007
Games loaded...21500008
Games loaded...21500009
Games loaded...21500010
Games loaded...21500011
Games loaded...21500012
Games loaded...21500013
Games loaded...21500014
Games loaded...21500015
Games loaded...21500016
Games loaded...21500017
Games loaded...21500018
Games loaded...21500019
Games loaded...21500020
Games loaded...21500021
Games loaded...21500022
Games loaded...21500023
Games loaded...21500024
Games loaded...21500025
Games loaded...21500026
Games loaded...21500027
Games loaded...21500028
Games loaded...21500029
Games loaded...21500030
Games loaded...21500031
Games loaded...21500032
Games loaded...21500033
Games loaded...21500034
Games loaded...21500035
Games loaded...21500036
Games loaded...21500037
Games loaded...21500038
Games loaded...21500039
Games loaded...21500040
Games loaded...21500041
Games loaded...2

Games loaded...21500343
Games loaded...21500344
Games loaded...21500345
Games loaded...21500346
Games loaded...21500347
Games loaded...21500348
Games loaded...21500349
Games loaded...21500350
Games loaded...21500351
Games loaded...21500352
Games loaded...21500353
Games loaded...21500354
Games loaded...21500355
Games loaded...21500356
Games loaded...21500357
Games loaded...21500358
Games loaded...21500359
Games loaded...21500360
Games loaded...21500361
Games loaded...21500362
Games loaded...21500363
Games loaded...21500364
Games loaded...21500365
Games loaded...21500366
Games loaded...21500367
Games loaded...21500368
Games loaded...21500369
Games loaded...21500370
Games loaded...21500371
Games loaded...21500372
Games loaded...21500373
Games loaded...21500374
Games loaded...21500375
Games loaded...21500376
Games loaded...21500377
Games loaded...21500378
Games loaded...21500379
Games loaded...21500380
Games loaded...21500381
Games loaded...21500382
Games loaded...21500383
Games loaded...2

Games loaded...21500685
Games loaded...21500686
Games loaded...21500687
Games loaded...21500688
Games loaded...21500689
Games loaded...21500690
Games loaded...21500691
Games loaded...21500692
Games loaded...21500693
Games loaded...21500694
Games loaded...21500695
Games loaded...21500696
Games loaded...21500697
Games loaded...21500698
Games loaded...21500699
Games loaded...21500700
Games loaded...21500701
Games loaded...21500702
Games loaded...21500703
Games loaded...21500704
Games loaded...21500705
Games loaded...21500706
Games loaded...21500707
Games loaded...21500708
Games loaded...21500709
Games loaded...21500710
Games loaded...21500711
Games loaded...21500712
Games loaded...21500713
Games loaded...21500714
Games loaded...21500715
Games loaded...21500716
Games loaded...21500717
Games loaded...21500718
Games loaded...21500719
Games loaded...21500720
Games loaded...21500721
Games loaded...21500722
Games loaded...21500723
Games loaded...21500724
Games loaded...21500725
Games loaded...2

Games loaded...21501028
Games loaded...21501029
Games loaded...21501030
Games loaded...21501031
Games loaded...21501032
Games loaded...21501033
Games loaded...21501034
Games loaded...21501035
Games loaded...21501036
Games loaded...21501037
Games loaded...21501038
Games loaded...21501039
Games loaded...21501040
Games loaded...21501041
Games loaded...21501042
Games loaded...21501043
Games loaded...21501044
Games loaded...21501045
Games loaded...21501046
Games loaded...21501047
Games loaded...21501048
Games loaded...21501049
Games loaded...21501050
Games loaded...21501051
Games loaded...21501052
Games loaded...21501053
Games loaded...21501054
Games loaded...21501055
Games loaded...21501056
Games loaded...21501057
Games loaded...21501058
Games loaded...21501059
Games loaded...21501060
Games loaded...21501061
Games loaded...21501062
Games loaded...21501063
Games loaded...21501064
Games loaded...21501065
Games loaded...21501066
Games loaded...21501067
Games loaded...21501068
Games loaded...2

Games loaded...21600141
Games loaded...21600142
Games loaded...21600143
Games loaded...21600144
Games loaded...21600145
Games loaded...21600146
Games loaded...21600147
Games loaded...21600148
Games loaded...21600149
Games loaded...21600150
Games loaded...21600151
Games loaded...21600152
Games loaded...21600153
Games loaded...21600154
Games loaded...21600155
Games loaded...21600156
Games loaded...21600157
Games loaded...21600158
Games loaded...21600159
Games loaded...21600160
Games loaded...21600161
Games loaded...21600162
Games loaded...21600163
Games loaded...21600164
Games loaded...21600165
Games loaded...21600166
Games loaded...21600167
Games loaded...21600168
Games loaded...21600169
Games loaded...21600170
Games loaded...21600171
Games loaded...21600172
Games loaded...21600173
Games loaded...21600174
Games loaded...21600175
Games loaded...21600176
Games loaded...21600177
Games loaded...21600178
Games loaded...21600179
Games loaded...21600180
Games loaded...21600181
Games loaded...2

Games loaded...21600483
Games loaded...21600484
Games loaded...21600485
Games loaded...21600486
Games loaded...21600487
Games loaded...21600488
Games loaded...21600489
Games loaded...21600490
Games loaded...21600491
Games loaded...21600492
Games loaded...21600493
Games loaded...21600494
Games loaded...21600495
Games loaded...21600496
Games loaded...21600497
Games loaded...21600498
Games loaded...21600499
Games loaded...21600500
Games loaded...21600501
Games loaded...21600502
Games loaded...21600503
Games loaded...21600504
Games loaded...21600505
Games loaded...21600506
Games loaded...21600507
Games loaded...21600508
Games loaded...21600509
Games loaded...21600510
Games loaded...21600511
Games loaded...21600512
Games loaded...21600513
Games loaded...21600514
Games loaded...21600515
Games loaded...21600516
Games loaded...21600517
Games loaded...21600518
Games loaded...21600519
Games loaded...21600520
Games loaded...21600521
Games loaded...21600522
Games loaded...21600523
Games loaded...2

Games loaded...21600826
Games loaded...21600827
Games loaded...21600828
Games loaded...21600829
Games loaded...21600830
Games loaded...21600831
Games loaded...21600832
Games loaded...21600833
Games loaded...21600834
Games loaded...21600835
Games loaded...21600836
Games loaded...21600837
Games loaded...21600838
Games loaded...21600839
Games loaded...21600840
Games loaded...21600841
Games loaded...21600842
Games loaded...21600843
Games loaded...21600844
Games loaded...21600845
Games loaded...21600846
Games loaded...21600847
Games loaded...21600848
Games loaded...21600849
Games loaded...21600850
Games loaded...21600851
Games loaded...21600852
Games loaded...21600853
Games loaded...21600854
Games loaded...21600855
Games loaded...21600856
Games loaded...21600857
Games loaded...21600858
Games loaded...21600859
Games loaded...21600860
Games loaded...21600861
Games loaded...21600862
Games loaded...21600863
Games loaded...21600864
Games loaded...21600865
Games loaded...21600866
Games loaded...2

In [ ]:
data_model = data_clean.drop(labels=['sequence_id', 'period', 'play_clock', 'home_description', 'away_description', 'score', 'player1_id', 'player1_name', 'player1_team', 'player2_id', 'player2_name', 'player2_team', 'player3_id', 'player3_name','player3_team', 'event_type', 'home', 'away', 'event_description'], axis=1)
data_model.head()

In [ ]:
data_model.replace([np.inf, -np.inf], np.nan, inplace=True)
data_model.fillna(value=0, inplace=True)
data_clean.columns

In [ ]:
data_model.describe()

In [42]:
# Saving the file
filename = "data_clean.csv"
data_clean.to_csv(filename)
# filename = "data_model.csv"
# data_model.to_csv(filename)
    